# Phase 1 - Model Testing
## Package Import

In [7]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


## Data Import

In [10]:
df = pd.read_csv('../../data/ETF_cleaned.csv')
print(df.columns)
df.head()

Index(['Unnamed: 0', 'fund_symbol', 'quote_type', 'region', 'fund_short_name',
       'fund_long_name', 'currency', 'fund_category', 'fund_family',
       'exchange_code', 'exchange_name', 'exchange_timezone', 'avg_vol_3month',
       'avg_vol_10day', 'total_net_assets', 'day50_moving_average',
       'day200_moving_average', 'week52_high_low_change',
       'week52_high_low_change_perc', 'week52_high', 'week52_high_change',
       'week52_high_change_perc', 'week52_low', 'week52_low_change',
       'week52_low_change_perc', 'fund_yield', 'inception_date',
       'annual_holdings_turnover', 'investment_type', 'size_type',
       'fund_annual_report_net_expense_ratio',
       'category_annual_report_net_expense_ratio', 'asset_stocks',
       'asset_bonds', 'fund_sector_basic_materials',
       'fund_sector_communication_services', 'fund_sector_consumer_cyclical',
       'fund_sector_consumer_defensive', 'fund_sector_energy',
       'fund_sector_financial_services', 'fund_sector_healthca

,Unnamed: 0,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,fund_category,fund_family,exchange_code,...,category_return_2009,category_return_2008,category_return_2007,fund_alpha_3years,fund_beta_3years,fund_mean_annual_return_3years,fund_r_squared_3years,fund_stdev_3years,fund_sharpe_ratio_3years,fund_treynor_ratio_3years
0,0,AAAU,ETF,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,NaN,DWS,PCX,...,0.32673,-0.39004,0.08189,13.180,0.07,1.23,0.54,14.93,0.91,187.10
1,1,AADR,ETF,US,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,USD,Foreign Large Growth,Virtus,NGM,...,0.29080,-0.42427,0.31193,-1.300,1.11,0.85,75.96,22.42,0.40,6.11
2,2,AAXJ,ETF,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,Pacific/Asia ex-Japan Stk,American Century Investments,NGM,...,0.73761,-0.51342,0.39069,1.200,0.90,0.80,74.34,18.48,0.46,7.80
3,3,ABEQ,ETF,US,Thrivent Large Cap Growth Fund,Thrivent Large Cap Growth Fund Class A,USD,Large Value,Thrivent Funds,PCX,...,0.25157,-0.35846,0.01809,-0.715,1.02,0.88,78.01,19.66,0.56,8.40
4,4,ACES,ETF,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,Miscellaneous Sector,American Century Investments,PCX,...,0.31262,-0.33986,0.01815,24.180,1.31,3.65,52.02,32.60,1.30,34.50


## Data Preprocessing

## Model 1: Erdős-Rényi network

In [ ]:
n = 100  # number of nodes
p = 0.1  # probability

# network generation
G_er = nx.erdos_renyi_graph(n, p)

nx.draw(G_er, with_labels=True, node_size=500, node_color="skyblue")
plt.title("Erdős-Rényi Network")
plt.show()

## Model 2: Preferential Attachment Model

In [ ]:
m = 3  # number of edges to attach from a new node to existing nodes

G_pa = nx.barabasi_albert_graph(n, m)

# Draw the graph
nx.draw(G_pa, with_labels=True, node_size=500, node_color="lightgreen")
plt.title("Preferential Attachment Model Network")
plt.show()

# get centralities
# use to caluclate risk

## Model 3: Neural Network Model

In [ ]:
# Data to tensors
X = torch.tensor(X, dtype=torch.float32).view(-1, 1)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
loader = DataLoader(df, batch_size=32, shuffle=True)

# train/test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# get device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# define NN class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#instantiate model
model = NeuralNetwork().to(device)

# loss function, optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# test the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test).numpy()

# calculate predicted value at risk
predicted_var = np.percentile(y_pred, 95)
